# Carga Gold - Fato Carteira

Este notebook realiza a carga da fato de cotação (fato_cotacao) a partir dos dados da tabela staging de controle ativo.

## Imports

In [27]:
from spark_config import init_spark
from pyspark.sql import SparkSession, DataFrame
from pyspark.sql import functions as F
from datetime import datetime
from dateutil.relativedelta import relativedelta
from functools import reduce

## Start Spark Session

In [28]:
spark = init_spark("Carga fato carteira")


✅ Spark 3.5.7 iniciado com Hive local persistente!
📁 Warehouse: D:/Projetos/DataLake/spark-warehouse
📁 Metastore: D:/Projetos/DataLake/metastore_db



## Define Delta Table Paths

In [29]:
# Define caminhos locais onde serão armazenadas as tabelas Delta
base_silver_path = "D:/Projetos/Jornada_financas_pessoais/data/delta/silver"
base_gold_path = "D:/Projetos/Jornada_financas_pessoais/data/delta/gold"

# Define caminhos das tabelas Delta
delta_path_controle_ativo = f"{base_silver_path}/stg_controle_ativo"
delta_path_fato_carteira = f"{base_gold_path}/fato_carteira"

## Define Functions

In [30]:
"""
Pipeline para carregar posição mensal de carteira de investimentos
Abordagem funcional
"""
def extrair_operacoes_mes(spark: SparkSession, mes_ref_base: str, cpf: str = None) -> DataFrame:
    """
    Extrair operações do mês da tabela silver
    
    Args:
        spark: SparkSession
        mes_ref_base: Mês de referência base no formato 'YYYY-MM'
        cpf: CPF do investidor (opcional)
    """
    query = f"""
    SELECT 
        mes_referencia,
        '{mes_ref_base}' AS mes_ref_base,
        cpf,
        cotista,
        cd_ativo,
        dt_operacao,
        cd_tipo_operacao,
        qt_operacao,
        vl_preco_ativo,
        vl_custo_total,
        vl_rateio,
        qt_estoque,
        vl_pmedio
    FROM silver.stg_controle_ativo
    WHERE mes_referencia <= '{mes_ref_base}'
    """
    
    if cpf:
        query += f" AND cpf = '{cpf}'"
        
    return spark.sql(query)


def extrair_cotacoes_mes(spark: SparkSession) -> DataFrame:
    """
    Extrair cotações do último dia do mês da tabela gold
    
    Args:
        spark: SparkSession
        mes_ref_base: Mês de referência base no formato 'YYYY-MM'
    """
    query = f"""
    SELECT 
        DATE_FORMAT(dt_pregao, 'yyyy-MM') AS mes_ref_base,
        dt_pregao,
        cd_ativo,
        vl_medio AS vl_ativo
    FROM gold.fato_cotacao t1
    INNER JOIN gold.dim_tempo t2 ON t1.dt_pregao = t2.dt_dia
    INNER JOIN gold.dim_ativo_financeiro t3 ON t1.sk_ativo = t3.sk_ativo
    WHERE t1.dt_pregao  = t2.dt_ultimo_dia_util_mes 
    """
        
    return spark.sql(query)


def calcular_posicao_final(df_operacoes: DataFrame, df_cotacoes: DataFrame) -> DataFrame:
    """
    Calcula a posição final de cada ativo no mês.
    Considera múltiplas operações no mesmo mês e ajusta estoque conforme tipo de operação.
    """
    # Ajusta a quantidade: vendas como negativas
    df_ajustado = (
        df_operacoes
        .withColumn("qt_ajustada",
            F.when(F.col("cd_tipo_operacao").isin("V", "VENDA"), -F.col("qt_operacao"))
            .otherwise(F.col("qt_operacao"))
        )
        .withColumn("vl_total_operacao_ajustada",
            (F.col("qt_ajustada") * F.col("vl_preco_ativo")) + F.col("vl_rateio")
        )
    )

    # Agrupa por ativo, cotista e mês de referência
    df_agrupado = df_ajustado.groupBy(
        "mes_ref_base",
        "cpf",
        "cotista",
        "cd_ativo"
    ).agg(
        F.sum("qt_ajustada").alias("qt_estoque"),
        (F.sum("vl_total_operacao_ajustada") / F.sum("qt_ajustada")).alias("vl_pmedio"),
    )

    # Join com cotações por cd_ativo e mes_ref_base (ano_mes)
    df_agrupado = df_agrupado.join(
        df_cotacoes,
        on=["cd_ativo", "mes_ref_base"],
        how="inner" 
    )

    # Calcula o valor da posição
    df_resultado = (
        df_agrupado
        .withColumn("vl_investido", F.col("qt_estoque") * F.col("vl_pmedio"))
        .withColumn("vl_carteira", F.col("qt_estoque") * F.col("vl_ativo"))
    )

    return df_resultado

## Read Source Data

In [31]:
# Extrair a menor e maior data de operação da tabela de controle
print("Identificando o período de operações...")
df_controle = spark.table("silver.stg_controle_ativo")

# Buscar min e max em uma única operação (mais eficiente)
min_max = df_controle.agg(
    F.min("dt_operacao").alias("min_date"),
    F.max("dt_operacao").alias("max_date")
).collect()[0]

min_date = min_max["min_date"]
max_date = min_max["max_date"]

print(f"   → Menor data encontrada: {min_date}")
print(f"   → Maior data encontrada: {max_date}")

# Gerar lista de meses a processar (desde a menor até a maior data)
data_inicio = datetime.strptime(str(min_date), "%Y-%m-%d")
data_fim = datetime.strptime(str(max_date), "%Y-%m-%d")

meses_processar = []
data_atual = data_inicio
while data_atual <= data_fim:
    meses_processar.append(data_atual.strftime("%Y-%m"))
    data_atual += relativedelta(months=1)

print(f"   → {len(meses_processar)} meses para processar: {meses_processar[0]} até {meses_processar[-1]}")


Identificando o período de operações...
   → Menor data encontrada: 2024-09-23
   → Maior data encontrada: 2025-07-29
   → 11 meses para processar: 2024-09 até 2025-07


In [32]:
# Lista para acumular os DataFrames de cada mês
lista_posicoes = []

df_cotacoes = extrair_cotacoes_mes(spark)

print(f"\n{'='*40}")

# Loop por cada mês
for mes_referencia in meses_processar:
    print(f"Processando mês: {mes_referencia}")
    
    # 1. Extrair operações
    df_operacoes = extrair_operacoes_mes(spark, mes_referencia)
    count_operacoes = df_operacoes.count()
    
    if count_operacoes == 0:
        print("⚠ Nenhuma operação encontrada para o período")
        continue
    
    # 2. Calcular posição final por ativo
    df_posicao = calcular_posicao_final(df_operacoes, df_cotacoes)
    
    # Acumular o DataFrame na lista
    lista_posicoes.append(df_posicao)

print(f"{'='*40}")


Processando mês: 2024-09
Processando mês: 2024-10
Processando mês: 2024-11
Processando mês: 2024-12
Processando mês: 2025-01
Processando mês: 2025-02
Processando mês: 2025-03
Processando mês: 2025-04
Processando mês: 2025-05
Processando mês: 2025-06
Processando mês: 2025-07


## Transform Data

In [33]:
# Unir todos os DataFrames de forma eficiente usando reduce
df_union = reduce(lambda df1, df2: df1.unionByName(df2), lista_posicoes)
    
total_registros = df_union.count()
print(f"   → Total de {total_registros} registros a serem gravados")

   → Total de 98 registros a serem gravados


In [34]:
# Leitura das dimensões
df_dim_cliente = spark.table("gold.dim_cliente")
df_dim_ativo = spark.table("gold.dim_ativo_financeiro")

# Join com as duas dimensões
df_joined = (
    df_union.alias("stg")
    .join(
        df_dim_ativo.alias("dim_ativo"),
        F.col("stg.cd_ativo") == F.col("dim_ativo.cd_ativo"),
        "left"
    )
    .join(
        df_dim_cliente.alias("dim_cliente"),
        F.col("stg.cpf") == F.col("dim_cliente.cd_cpf_pessoa"),
        "left"
    )
)

# Tratamento da chave surrogate faltante (usa -1)
df_fato_carteira = (
    df_joined.select(
        F.col("stg.dt_pregao").alias("dt_carteira"),
        F.when(F.col("sk_cliente").isNull(), F.lit("-1")).otherwise(F.col("sk_cliente")).alias("sk_cliente"),
        F.when(F.col("sk_ativo").isNull(), F.lit("-1")).otherwise(F.col("sk_ativo")).alias("sk_ativo"),
        F.col("qt_estoque").alias("qt_ativo"),
        F.col("vl_ativo"),
        F.col("vl_pmedio"),
        F.col("vl_investido"),
        F.col("vl_carteira"),
        F.current_timestamp().alias("ts_insercao")
    )
)

## Write Data

In [35]:

df_fato_carteira.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "false") \
    .option("partitionOverwriteMode", "dynamic") \
    .partitionBy("sk_cliente") \
    .save(delta_path_fato_carteira)

print("✅ Dynamic partition overwrite executado - apenas partições afetadas foram sobrescritas")

✅ Dynamic partition overwrite executado - apenas partições afetadas foram sobrescritas


## Stop Spark Session

In [36]:
# Encerra a SparkSession
spark.stop()